In [1]:
!pip install tensorflow keras_cv --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 30.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.


In [2]:
import keras_cv
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
from PIL import Image

# Enable mixed precision
# (only do this if you have a recent NVIDIA GPU)
keras.mixed_precision.set_global_policy("mixed_float16")

# Instantiate the Stable Diffusion model
model = keras_cv.models.StableDiffusion(jit_compile=True)

By using this model checkpoint, you acknowledge that its usage is subject to the terms of the CreativeML Open RAIL-M license at https://raw.githubusercontent.com/CompVis/stable-diffusion/main/LICENSE


In [3]:
fps = 60 #@param{type:'slider', min:5, max:60, step:5}
rubber_band = True #@param{type:'boolean'}

def export_as_gif(filename, images, frames_per_second=fps, rubber_band=rubber_band):
    if rubber_band:
        images += images[2:-1][::-1]
    images[0].save(
        filename,
        save_all=True,
        append_images=images[1:],
        duration=1000 // frames_per_second,
        loop=0,
    )

In [4]:

fps = 60 #@param{type:'slider', min:5, max:60, step:5}
rubber_band = True #@param{type:'boolean'}

from google.colab import drive
drive.mount('/content/drive')

def export_as_gif_to_drive(filename, images, frames_per_second=10, rubber_band=False):
    if rubber_band:
        images += images[2:-1][::-1]
    images[0].save(
        '/content/drive/My Drive/' + filename,  # Save to Google Drive
        save_all=True,
        append_images=images[1:],
        duration=1000 // frames_per_second,
        loop=0,
    )


Mounted at /content/drive


In [5]:
def git_filepath(file_name):
    file_format = ".gif"
    saving_dir = f"{file_name}{file_format}"
    return saving_dir

In [6]:
# Note that in order to maintain some stability between the resulting images we
# keep the diffusion noise constant between images.

seed = 10000 #@param{type:'integer'}
noise = tf.random.normal((512 // 8, 512 // 8, 4), seed=seed)



In [7]:
prompt = "Transform a redundant London street into a vibrant green streetscape! Envision planters adorning buildings, cyclists navigating wide footpaths, lush greenery enhancing the environment, and pedestrian-friendly streets bustling with life. Create a visual narrative of sustainable urban diffusion, depicting a harmonious blend of nature, community, and active living" #@param{type:'string'}
encoding = tf.squeeze(model.encode_text(prompt))
walk_steps = 6 #@param{type:'integer'}
batch_size = 3
batches = walk_steps // batch_size
num_steps = 25 #@param{type:'slider', min:0, max:100, step:5}

walk_noise_x = tf.random.normal(noise.shape, dtype=tf.float64)
walk_noise_y = tf.random.normal(noise.shape, dtype=tf.float64)

walk_scale_x = tf.cos(tf.linspace(0, 2, walk_steps) * math.pi)
walk_scale_y = tf.sin(tf.linspace(0, 2, walk_steps) * math.pi)
noise_x = tf.tensordot(walk_scale_x, walk_noise_x, axes=0)
noise_y = tf.tensordot(walk_scale_y, walk_noise_y, axes=0)
noise = tf.add(noise_x, noise_y)
batched_noise = tf.split(noise, batches)

images = []
for batch in range(batches):
    images += [
        Image.fromarray(img)
        for img in model.generate_image(
            encoding,
            batch_size=batch_size,
            num_steps=num_steps,
            diffusion_noise=batched_noise[batch],
        )
    ]

export_as_gif("walk_around.gif", images)

1356917/1356917 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
492466864/492466864 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step
3439090152/3439090152 ━━━━━━━━━━━━━━━━━━━━ 65s 0us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 4703s 180s/step
198180272/198180272 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 4462s 179s/step


In [8]:
file_name = 'walk_around_2.gif' #@param{type:'string'}
export_as_gif(file_name, images, rubber_band=True)

In [9]:
from IPython.display import Image as IImage
IImage(file_name)

Output hidden; open in https://colab.research.google.com to view.